In [ ]:
import backtrader as bt

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# 下面为设置现金代码，cerebro.set_cash(1000000)
import backtrader as bt
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100000.0)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# 下面为数据输入 data feeds
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
# Import the backtrader platform
import backtrader as bt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2000, 12, 31),
        reverse=False)
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # Run over everything
    cerebro.run()
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# 有数据 data feeds 后， 策略消费处理数据，生成交易决策信号
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
# Import the backtrader platform
import backtrader as bt
# Create a Stratey
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2000, 12, 31),
        # Do not pass values after this date
        reverse=False)
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # Run over everything
    cerebro.run()
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# 完善交易策略， 在函数next下面写交易策略，上涨3天卖出，连续下跌3天买入。

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
# Import the backtrader platform
import backtrader as bt
# Create a Stratey
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close
            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2000, 12, 31),
        # Do not pass values after this date
        reverse=False)
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # Run over everything
    cerebro.run()
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# 完善买卖点，什么时候买的，需要定义交易状态检测函数如下
def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        # Write down: no pending order
        self.order = None

In [1]:
# 完整代码 

#交易策略为连续下跌3天买入，连续上涨3天卖出
import pandas as pd
import matplotlib.pyplot as plt
import backtrader as bt
import backtrader.feeds as btfeed
import backtrader.indicators as btind
from backtrader import analyzers
import pyalgotrade
import  PySimpleGUI as sg
import akshare as ak
import pandas as pd
import datetime

%matplotlib inline

stock=sg.popup_get_file('输入股票代码比如sz002466')
start_date=sg.popup_get_file('输入数据开始时间比如20200101')
start_cash=sg.popup_get_file('输入开始资金比如1000000')

#定义测试类
class testStrategy(bt.Strategy):
    #定义记录函数
    def log(self,txt,dt=None):
        dt=dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    #初始化数据
    def __init__(self):
        self.dataclose=self.datas[0].close
    #定义交易状态检测函数，交易前
    def notify_order(self,order):
        #如果交易提交/接受
        if order.status in [order.Submitted,order.Accepted]:
            print('交易提交，交易接受')
        #如果交易完成
        if order.status in [order.Completed]:
            #如果交易类型是买
            if order.isbuy():
                self.log('买的价格， %.2f' % order.executed.price)
            #如果交易类型是卖
            elif order.issell():
                self.log('卖的价格, %.2f' % order.executed.price)
            self.bar_executed=len(self)
        #如果交易交易取消，保证金不足，交易拒绝
        elif order.status in [order.Canceled,order.Margin,order.Rejected]:
          self.log('交易取消/保证金不足/交易拒绝')  
        self.order=None
    #主交易函数
    def next(self):
        self.log('收盘价,%.2f' % self.dataclose[0])
        #写交易函数，连续下跌3天买入,连续上涨3天卖出
        #如果今天的价格小于昨天的价格
        if self.dataclose[0]<self.dataclose[-1]:
            #如果昨天的价格小于前天的价格
            if self.dataclose[-1]<self.dataclose[-2]:
                #记录买卖的价格
                self.log('买的价格 %.2f' % self.dataclose[0])
                self.buy(size=200)#单位为股，代表1手
        elif self.dataclose[0]>self.dataclose[-1]:
            if self.dataclose[-1]>self.dataclose[-2]:
                self.log('卖出的价格 %.2f' %self.dataclose[0])
                self.sell(size=100)
#简单例子
if __name__ == "__main__":
    #将大脑实例化
    cerebro=bt.Cerebro()
    cerebro.addstrategy(testStrategy)
    #加入数据
    df=ak.stock_zh_a_daily(symbol=stock,start_date=start_date)
    df.index=pd.to_datetime(df['date'])
    data=btfeed.PandasData(dataname=df)
    cerebro.adddata(data=data)
    #设置开始资金
    cerebro.broker.set_cash(int(start_cash))
    #设置交易费用
    cerebro.broker.setcommission(0.003)
    print('开始值{}'.format(cerebro.broker.getvalue()))
    cerebro.run()
    print('最终值{}'.format(cerebro.broker.getvalue()))
    cerebro.plot(style='candle')
    plt.show()

开始值1000000
2020-01-02, 收盘价,30.69
2020-01-02, 买的价格 30.69
交易提交，交易接受
交易提交，交易接受
2020-01-03, 买的价格， 29.80
2020-01-03, 收盘价,30.95
2020-01-06, 收盘价,32.68
2020-01-06, 卖出的价格 32.68
交易提交，交易接受
交易提交，交易接受
2020-01-07, 卖的价格, 32.18
2020-01-07, 收盘价,32.21
2020-01-08, 收盘价,31.94
2020-01-08, 买的价格 31.94
交易提交，交易接受
交易提交，交易接受
2020-01-09, 买的价格， 32.24
2020-01-09, 收盘价,31.53
2020-01-09, 买的价格 31.53
交易提交，交易接受
交易提交，交易接受
2020-01-10, 买的价格， 31.41
2020-01-10, 收盘价,32.00
2020-01-13, 收盘价,32.01
2020-01-13, 卖出的价格 32.01
交易提交，交易接受
交易提交，交易接受
2020-01-14, 卖的价格, 32.80
2020-01-14, 收盘价,34.78
2020-01-14, 卖出的价格 34.78
交易提交，交易接受
交易提交，交易接受
2020-01-15, 卖的价格, 34.10
2020-01-15, 收盘价,33.07
2020-01-16, 收盘价,32.76
2020-01-16, 买的价格 32.76
交易提交，交易接受
交易提交，交易接受
2020-01-17, 买的价格， 32.85
2020-01-17, 收盘价,32.23
2020-01-17, 买的价格 32.23
交易提交，交易接受
交易提交，交易接受
2020-01-20, 买的价格， 32.20
2020-01-20, 收盘价,32.58
2020-01-21, 收盘价,31.75
2020-01-22, 收盘价,32.56
2020-01-23, 收盘价,30.82
2020-02-03, 收盘价,27.74
2020-02-03, 买的价格 27.74
交易提交，交易接受
交易提交，交易接受
2020-02-04, 买的价格， 24.97
2020-02-0

<IPython.core.display.Javascript object>